In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

get_ipython().magic('matplotlib inline')
get_ipython().magic('reload_ext autoreload')
get_ipython().magic('autoreload 2')

## Unit Testing

In [ ]:
from streetstyle_dataset import StreetStyleDataset
import data_utils

transform = transforms.Compose([
        data_utils.ResizeTransform(299),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
dataset = StreetStyleDataset('../data/streetstyle27k', '../data', transform=transform)

def imshow(inp):
    """Imshow for Tensor."""
    plt.figure()
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)

images, labels = dataset.next_train()
img = torchvision.utils.make_grid(images[:4], nrow=2)
imshow(img)

plt.figure()
images, labels = dataset.next_eval()
img = torchvision.utils.make_grid(images[:4], nrow=2)
imshow(img)

images, labels = dataset.next_test()
img = torchvision.utils.make_grid(images[:4], nrow=2)
imshow(img)

In [ ]:
from classifier_model import StreetStyleClassifier
from torch.autograd import Variable

imag_var = Variable(images)
model = StreetStyleClassifier()
model.set_eval_attributes([True, True, False, True, False, False,False,False,False,False,False,False])
out = model(imag_var)
print(out)

# Model Training

In [ ]:
import torch
import torch.nn as nn
import torchvision
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

get_ipython().magic('matplotlib inline')
get_ipython().magic('reload_ext autoreload')
get_ipython().magic('autoreload 2')

from classifier_test import StreetStyleClassifierTest

In [ ]:
test = StreetStyleClassifierTest(use_gpu=True)

In [ ]:
test.create_data_loaders()

In [ ]:
test.visualize_single_batch()

In [ ]:
def get_num_parameters(model):
    total_num=0
    if type(model) == type(dict()):
        for key in model:
            for p in model[key].parameters():
                total_num+=p.nelement()
    else:
        for p in model.parameters():
            total_num+=p.nelement()
    return total_num

test.create_model()
print(test.model)
print('num params: ' + str(get_num_parameters(test.model)))

In [ ]:
test.create_loss_function()

In [ ]:
test.create_optimizer()

In [ ]:
kwargs = {}
kwargs["visualize_every_n_batches"] = 50
kwargs["save_every_n_batches"] = 200
test.train_model(num_iters=10000, **kwargs)

In [ ]:
# load best model from saved checkpoint
checkpoint_name = './log/1519285290_2600.tar'
log=torch.load(checkpoint_name)
model=log['best_model']
test.log = log
test.model = model

In [ ]:
# plot the losses and accuracies
plt.figure()
matplotlib.rcParams['font.size'] = 14
for key in test.log:
    if 'val_mean_class_acc' == key and test.log[key] is not None:
        y = [np.sum(x) for x in test.log[key]]
        x = np.arange(0, 2601, 200)
        plt.plot(x, y, label=key)
        plt.legend()
        plt.ylabel('Summed MCA')
        plt.xlabel('Iterations')
        plt.show()

In [ ]:
running_loss, running_correct, test_mean_class_acc = test.test_model()

In [ ]:
print('Test Loss: ' + str(running_loss))
print('Test Accuracy: ' + str(running_correct))
print('Test MCA: ' + str(test_mean_class_acc))